In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
os.chdir('/content/drive/MyDrive/vietnamesesentiment ')

In [3]:
!pip install transformers

     |████████████████████████████████| 2.0MB 18.8MB/s 
     |████████████████████████████████| 3.2MB 38.5MB/s 
     |████████████████████████████████| 890kB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=1d32a8057311756498a15ffee962af68503f6e33fd0b6f4d0aa2ca66ea6383fe
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

from transformers import *
import numpy as np
import argparse
from tqdm import tqdm 
import random
import transformers
import pandas as pd
from collections import Counter
from transformers import get_linear_schedule_with_warmup
import sys
sys.argv=['']
del sys

In [5]:
df = pd.read_csv('combine.csv')
X = df['sentence'].values
Y = df['label'].values
index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
Y = Y[index]

In [ ]:
!pip install unidecode

     |████████████████████████████████| 245kB 4.3MB/s 


In [7]:
import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

In [8]:
def transform(x):
  return [s.lower() for s in x] 
def remove_duplicate(x):
  result = []
  for s in x:
      s = re.sub(r'([a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      s = re.sub(r'([a-z][a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      result.append(s)
  return result

In [9]:
X = transform(X)
X = remove_duplicate(X)

In [10]:
X[0]

'mình đến quán lúc 18h30 lúc này quán khá vắng . .\ncó điều bực mình có mấy đứa nhóc cấp 2 đến trước mở nhạc hàn xẻng làm mình đau hết cả đầu ... đồ ăn thì đưa ra hơi bị chậm ... : (\nns chung ổn . .\nlần 2 đến quán . .'

In [11]:
X[1]

'quán nhỏ xinh , nhân viên đc . lúc đầu nếm kem chưa kĩ , thấy vị kem giống loteria , mà trong khi đó kem loteria có 3k/cây . nhưng tiền nào của đó , càng ăn thấy càng ngon , và vị có sự khác biệt rõ . kem tuơi , béo vừa và ngọt vừa . toping chất luợng , dâu đà lạt đàng hoàng , siro cũng dạng xịn . ăn phần kem strawbery 60k . ngon'

In [ ]:
text = ['Tôi tênn là Trần Nhật Quý']
print(remove_duplicate(text))

['Tôi tên là Trần Nhật Quý']


In [12]:
MAX_LEN = 120

parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=4)
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--maxlen', type=int, default=120)
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--lr', type=float, default=2e-5)
args = parser.parse_args()

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

MAX_LEN = args.maxlen

In [13]:
MAX_LEN

120

In [14]:
def get_dataloader(sentences, labels, tokenizer, batch_size):
    input_ids = []
    filtered_labels = []
    for i, sent in tqdm(enumerate(sentences)):
        encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
        if len(encoded_sent) <= 512:
            input_ids.append(encoded_sent)
            filtered_labels.append(labels[i])
    print(Counter(filtered_labels))
    inputs = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                                    value=0, truncating="post", padding="post")                                                     
    masks = []

    for sent in inputs:
        att_mask = [int(token_id > 0) for token_id in sent]
        masks.append(att_mask)

    inputs = torch.tensor(inputs)
    labels = torch.tensor(filtered_labels)
    masks = torch.tensor(masks)

    data = TensorDataset(inputs, masks, labels)
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

In [15]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [16]:
def get_baseline_acc(model, validation_dataloader, device):
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    eval_accuracy = eval_accuracy/nb_eval_steps
    return eval_accuracy

In [17]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [18]:
tokenizer = BertTokenizer.from_pretrained("FPTAI/vibert-base-cased", do_lower_case=True)
model = BertForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=3)
model.to(device)

Some weights of the model checkpoint at FPTAI/vibert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not in

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(38168, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
X_train = X[:-10000]
Y_train = Y[:-10000]
X_test = X[-10000:]
Y_test = Y[-10000:]
print(Counter(Y_train), Counter(Y_test))
train_dataloader = get_dataloader(X_train, Y_train, tokenizer, args.batch_size)
test_dataloader = get_dataloader(X_test, Y_test, tokenizer, args.batch_size)

optimizer = AdamW(model.parameters(),lr = args.lr)
total_steps = len(train_dataloader) * args.epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = int(0.1*total_steps),
                                            #warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
                                            #t_total = total_steps)

89it [00:00, 889.49it/s]

Counter({2: 11749, 0: 11008, 1: 355}) Counter({2: 5073, 0: 4767, 1: 160})


23112it [00:25, 920.37it/s]


Counter({2: 11748, 0: 11005, 1: 355})


10000it [00:10, 911.20it/s]


Counter({2: 5073, 0: 4764, 1: 160})


In [20]:
text = 'Trần Nhật Quý là thực tập sinh của công ti CMC'
tokenizer.encode(text)
tokenizer.decode(tokenizer.encode(text))

'[CLS] tran nhat quy la thuc tap sinh cua cong ti cmc [SEP]'

In [21]:
for epoch_i in range(args.epochs):
    total_loss = 0
    model.train()
        
    # For each batch of training data...
    for step, batch in tqdm(enumerate(train_dataloader)):
        model.train()
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
                
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss = outputs[0]
        loss.backward()

        total_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    test_accuracy = get_baseline_acc(model, test_dataloader, device)
    print(" Test acc {}".format( test_accuracy))

print("")
print("Training complete!")

2889it [10:01,  4.80it/s]



  Average training loss: 0.48


1it [00:00,  5.14it/s]

 Test acc 0.8680399999999999


2889it [10:00,  4.81it/s]


...


  Average training loss: 0.24


1it [00:00,  5.28it/s]

 Test acc 0.8698799999999999


2889it [09:54,  4.86it/s]



  Average training loss: 0.16
 Test acc 0.87164

Training complete!
